In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import osmnx as ox
import networkx as nx
from IPython.display import Image
%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

# POI

In [ ]:
import os
import pandas as pd
import requests
import json
import time
import random

from functools import partial

import pyproj
from shapely import geometry
from shapely.geometry import Point, Polygon
from shapely.ops import transform
import pickle


import folium

location_dict = {
    'Lindhagen': (59.3339639584289, 18.016154546711043),
    'Årsta': (59.29577094701488, 18.033270068215614),
    'Mörby': (59.39835855279321, 18.03616342969185),
    'Uppsala': (59.86299445509096, 17.645277270084502)
}


def get_polygon(center, radius, resolution):
    lat = center[0]
    lon = center[1]
    local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(
        lat, lon
    )
    
    
    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
        pyproj.Proj(local_azimuthal_projection),
    )
    
    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),    
    )
    
    center = Point(float(lon), float(lat))
    point_transformed = transform(wgs84_to_aeqd, center)
    
    #TODO: HERE FOR DIFFERENT SHAPE
    buffer = point_transformed.buffer(radius, resolution=resolution)
    
    
    # Get the polygon with lat lon coordinates
    polygon = transform(aeqd_to_wgs84, buffer)
    
    return polygon

In [ ]:
m = folium.Map([59.3321, 18.0666], zoom_start=10)
folium.LatLngPopup().add_to(m)

polygon = get_polygon(center=(59.3321, 18.0666), radius=500, resolution=10)

folium.GeoJson(polygon).add_to(m)

m

In [ ]:
#     'amenity': ['school', 'kindergarten', 'college', 'music_school', 'university'],
#     'amenity': ['bar', 'biergarten', 'pub'],
#     'amenity': ['clinic', 'hospital'],
#     'public_transport': ['station'], # big stations?
#     'railway': 'subway_entrance'
#     'railway': 'station' # pandatag, tunabana stations
#     'railway': 'tram_stop' # tran route 7 stops
#     'highway': 'bus_stop' # small bus stops



tags_traffic = {
    'railway': ['station', 'subway_entrance', 'tram_stop'],
    'highway': ['bus_stop']
}


tags_school = {
    'amenity': ['school', 'kindergarten']
}


tags_university = {
    'amenity': ['college', 'music_school', 'university']
}


tags_hospital = {
    'amenity': ['clinic', 'hospital']
}

tags_office = {
    'building': ['office']
}


tags_drink = {
    'amenity': ['bar', 'biergarten', 'pub']
}



tags_dict = {
    'Traffic': tags_traffic,
    'Kids': tags_school,
    'Student': tags_university,
    'Hospital': tags_hospital,
    'Office': tags_office,
    'Drink': tags_drink,
}


# RADIUS = 3000

counts = {}

for tag_name in tags_dict:
    print(tag_name)
    counts[tag_name] = []
    if tag_name in ['Traffic', 'Office', 'Drink']:
        RADIUS = 1000
    else:
        RADIUS = 3000
    for l_name in location_dict:
        print(l_name)
        tags = tags_dict[tag_name]
        location = location_dict[l_name]
        polygon = get_polygon(center=location, radius=RADIUS, resolution=40)
        gdf = ox.geometries_from_polygon(polygon=polygon, tags=tags)
        try:
            gdf = gdf.dropna(subset=['name']).dropna(axis=1, how='any')
            n = len(gdf)
        except:
            n = 0
        counts[tag_name].append(n)

        
df = pd.DataFrame(counts, index=location_dict.keys())

In [ ]:
df

In [ ]:
plt.figure()
plt.rcParams.update({'font.size': 24}) # must set in top

ax = df.plot(kind='bar', 
             title='Occasion',
             width=0.8,
        rot=45, 
        figsize= (18, 6), 
        subplots=False,
        legend=True
       )

ax.legend(fontsize=16)
ax.yaxis.grid(ls='--')
ax.set_ylabel('Counts', fontdict={'fontsize':24})

# Network

In [ ]:
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon

from IPython.display import Image


location_dict = {
    'Lindhagen': (59.3339639584289, 18.016154546711043),
    'Årsta': (59.29577094701488, 18.033270068215614),
    'Mörby': (59.39835855279321, 18.03616342969185),
    'Uppsala': (59.86299445509096, 17.645277270084502)
}

# location = (59.34397435059645, 18.01596439808984)
# location = (59.35397435059645, 18.01596439808984)


RADIUS = 3000
network_type = 'bike'
trip_times = [5, 10, 15]  # in minutes
# trip_times = [5, 10, 15, 20, 25]  # in minutes

travel_speed = 10  # biking speed in km/hour
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0, return_hex=True)

coverage_dict = {}

for l_name in location_dict:
    location = location_dict[l_name]
    polygon = get_polygon(center=location, radius=RADIUS, resolution=40)
    
    G = ox.graph_from_polygon(polygon=polygon, network_type=network_type)
    fig, ax = ox.plot_graph(G, node_size=0, figsize=(3, 3), save=True, filepath=f'{l_name}.png')
    center_node = ox.get_nearest_node(G, point=location)
    
    G = ox.project_graph(G)
    
    for _, _, _, data in G.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
        
    isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)

    fig, ax = ox.plot_graph(
        G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
    )
    coverage_dict[l_name] = []
    for polygon, fc, label in zip(isochrone_polys, iso_colors, sorted(trip_times, reverse=True)):
        patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.7, zorder=-1, label=f'{label} mins {network_type}')
        ax.add_patch(patch)
        coverage_dict[l_name].append(polygon.area)
    plt.legend()
    plt.title(f'{l_name}', fontsize=18)
    plt.savefig(f'{l_name}_iso.png')

In [ ]:
df = pd.DataFrame(coverage_dict, index=['15 mins bike', '10 mins bike', '5 mins bike'])

In [ ]:
plt.figure()
plt.rcParams.update({'font.size': 24}) # must set in top

ax = (df/1e6).plot(kind='bar', 
             title="Rider's coverage",
             width=0.8,
             rot=45, 
             figsize= (18, 6), 
             subplots=False,
             legend=True,
             color = ['tab:red', 'tab:green', 'tab:orange', 'tab:blue']
       )

ax.legend(fontsize=16)
ax.yaxis.grid(ls='--')
ax.set_ylabel('Counts', fontdict={'fontsize':24})

In [ ]:
# for l_name in location_dict:
#     location = location_dict[l_name]
#     polygon = get_polygon(center=location, radius=RADIUS, resolution=40)
    
#     G = ox.graph_from_polygon(polygon=polygon, network_type=network_type)
    
# #     G = ox.simplify_graph(G)

#     edge_centrality = nx.closeness_centrality(nx.line_graph(G))
#     nx.set_edge_attributes(G, edge_centrality, "edge_centrality")
    
#     ec = ox.plot.get_edge_colors_by_attr(G, "edge_centrality", cmap="inferno")
#     fig, ax = ox.plot_graph(G, edge_color=ec, edge_linewidth=2, node_size=0, save=True, filepath=f'{l_name}_centrality.png')

In [ ]:
entropy_dict = {}

for l_name in location_dict:
    print(l_name)
    location = location_dict[l_name]
    polygon = get_polygon(center=location, radius=1000, resolution=40)
    
    G = ox.graph_from_polygon(polygon=polygon, network_type=network_type)
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    entropy = ox.bearing.orientation_entropy(Gu)
    entropy = round(entropy, 2)
    entropy_dict[l_name] = entropy
    fig, ax = ox.bearing.plot_orientation(Gu, title=f'{l_name} (Entropy={entropy})', area=True)
    plt.savefig(f'{l_name}_orientation.png')

In [ ]:
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 24}) # must set in top


plt.title("The entropy of street orientation", fontsize=26)

plt.bar(range(len(entropy_dict)), entropy_dict.values(), color=['tab:red', 'tab:green', 'tab:orange', 'tab:blue'])
plt.xticks(np.arange(0, len(entropy_dict), 1), entropy_dict.keys(), fontsize=20, rotation=45)
plt.yticks(fontsize=20)
plt.ylabel("Entropy", fontsize=20)
plt.ylim([0, 5])
plt.grid(ls="--", axis="y")
plt.tight_layout()